In [ ]:
import numpy as np
import pandas as pd
import os
from multiprocessing import Pool
from tqdm import tqdm

from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore') 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

pd.set_option('expand_frame_repr', True) 
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.width', 180) 

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset

In [ ]:
from data_api import DataFeeder, DataAPI
from factor_base import FactorBase

# Instantiate the data interface
d = DataFeeder() 
api = DataAPI()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a custom Pearson correlation loss
class PearsonCorrelationLoss(nn.Module):
    def forward(self, y_pred, y_true):
        y_pred_mean = torch.mean(y_pred)
        y_true_mean = torch.mean(y_true)
        y_pred_std = torch.std(y_pred)
        y_true_std = torch.std(y_true)

        covariance = torch.mean((y_pred - y_pred_mean) * (y_true - y_true_mean))
        correlation = covariance / (y_pred_std * y_true_std)

        # Return the negative of Pearson correlation for minimization
        return -correlation

# Example GRU model
class GRUNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUNet, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output, _ = self.gru(x)
        output = self.linear(output[-1])
        return output

# Initialize model, loss, and optimizer
model = GRUNet(input_size, hidden_size, output_size)
criterion = PearsonCorrelationLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Example training loop
for epoch in range(epochs):
    for x_batch, y_batch in train_loader:
        # Forward pass
        y_pred = model(x_batch)
        loss = criterion(y_pred, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
